<a href="https://colab.research.google.com/github/dripshabh/machine-learning/blob/dripshabh-algorithms/MIT_Coursework_Week_2(Building%20and%20evaluating%20perceptrons).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#MIT 6.036 Spring 2019: Homework 2#

This colab notebook provides code and a framework for problems 7-10 of [the homework](https://openlearninglibrary.mit.edu/courses/course-v1:MITx+6.036+1T2019/courseware/Week2/week2_homework/1).  You can work out your solutions here, then submit your results back on the homework page when ready.

## <section>**Setup**</section>

First, download the code distribution for this homework that contains test cases and helper functions (such as `positive`).

Run the next code block to download and import the code for this lab.


In [3]:
!rm -f code_for_hw02.py*
!wget --no-check-certificate --quiet https://introml_oll.odl.mit.edu/6.036/static/homework/hw02/code_for_hw02.py
from code_for_hw02 import *

Importing code_for_hw02
New procedures added: tidy_plot, plot_separator, plot_data, plot_nonlin_sep, cv,
                      rv, y, positive, score
Data Sets: super_simple_separable_through_origin(), super_simple_separable(), xor(),
           xor_more()
Test data for problem 2.1: data1, labels1, data2, labels2
Test data for problem 2.2: big_data, big_data_labels, gen_big_data(), gen_lin_separable(),
                           big_higher_dim_separable(), gen_flipped_lin_separable()
Test functions: test_linear_classifier(), test_perceptron(), test_averaged_perceptron(),
                test_eval_classifier(), test_eval_learning_alg(), test_xval_learning_alg()

For more information, use 'help', e.g. 'help tidy_plot'
Done with import of code_for_hw02


In [4]:
help(tidy_plot)

Help on function tidy_plot in module code_for_hw02:

tidy_plot(xmin, xmax, ymin, ymax, center=False, title=None, xlabel=None, ylabel=None)
    Set up axes for plotting
    xmin, xmax, ymin, ymax = (float) plot extents
    Return matplotlib axes



In [5]:
def test(a):
  return a + 53

In [ ]:
def methodB(a):
  return test(a)

In [ ]:
def someMethod():
  test = 7
  return methodB(test + 3)

In [ ]:
someMethod()

63

# <section>**7) Implement perceptron**</section>

Implement [the perceptron algorithm](https://lms.mitx.mit.edu/courses/course-v1:MITx+6.036+2019_Spring/courseware/Week2/perceptron/2), where

* `data` is a numpy array of dimension $d$ by $n$
* `labels` is numpy array of dimension $1$ by $n$
* `params` is a dictionary specifying extra parameters to this algorithm; your algorithm should run a number of iterations equal to $T$
* `hook` is either None or a function that takes the tuple `(th, th0)` as an argument and displays the separator graphically.  We won't be testing this in the Tutor, but it will help you in debugging on your own machine.

It should return a tuple of $\theta$ (a $d$ by 1 array) and $\theta_0$ (a 1 by 1 array).

We have given you some  data sets in the code file for you to test your implementation.

Your function should initialize all parameters to 0, then run through the data, in the order it is given, performing an update to the parameters whenever the current parameters would make a mistake on that data point. Perform $T$ iterations through the data.

In [6]:
import numpy as np

In [19]:
def perceptron(data, labels, params = {}, hook = None):
    # if T not in params, default to 100
    T = params.get('T', 100)
    # Your implementation here
    d, n = data.shape
    theta = np.zeros((d,1))
    theta_0 = np.zeros(1)
    print("d = {}, n = {}, theta shape = {}, theta_0 shape = {}".format(d,n,theta.shape,theta_0.shape))

    for t in range(T):
      for i in range(n):
        y = labels[0,i]
        x = data[:,i]

        check = np.dot(theta.T, x) + theta_0
        #print(check)

        positive = np.sign(y*check) #set up expression to check sign of

        if np.sign(y*check) <= 0: #update thetas if there is a mistake
          theta[:,0] = theta[:,0] + y*x
          theta_0 = theta_0 + y
    print("shape x = {}, y = {}, theta = {}, theta_0 = {}".format(x.shape,y.shape,theta.shape,theta_0.shape))
    return (theta, theta_0)

In [20]:
test_perceptron(perceptron)

d = 2, n = 4, theta shape = (2, 1), theta_0 shape = (1,)
shape x = (2,), y = (), theta = (2, 1), theta_0 = (1,)
-----------Test Perceptron 0-----------
Passed! 

d = 2, n = 4, theta shape = (2, 1), theta_0 shape = (1,)
shape x = (2,), y = (), theta = (2, 1), theta_0 = (1,)
-----------Test Perceptron 1-----------
Passed! 



# <section>8) Implement averaged perceptron</section>

Regular perceptron can be somewhat sensitive to the most recent examples that it sees. Instead, averaged perceptron produces a more stable output by outputting the average value of `th` and `th0` across all iterations.

Implement averaged perceptron with the same spec as regular perceptron, and using the pseudocode below as a guide.
<pre>
procedure averaged_perceptron({(x^(i), y^(i)), i=1,...n}, T)
    th = 0 (d by 1); th0 = 0 (1 by 1)
    ths = 0 (d by 1); th0s = 0 (1 by 1)
    for t = 1,...,T do:
        for i = 1,...,n do:
	        if y^(i)(th . x^(i) + th0) <= 0 then
              th = th + y^(i)x^(i)
              th0 = th0 + y^(i)
	        ths = ths + th
	        th0s = th0s + th0
    return ths/(nT), th0s/(nT)
</pre>

In [9]:
def averaged_perceptron(data, labels, params={}, hook=None):
    # if T not in params, default to 100
    T = params.get('T', 100)
    # Your implementation here
    d, n = data.shape
    theta = np.zeros((d,1))
    theta_0 = np.zeros(1)
    ths = np.zeros((d,1))
    th0s = np.zeros(1)
    print("d = {}, n = {}, theta shape = {}, theta_0 shape = {}, ths shape = {}, th0s = {}".format(d,n,theta.shape,theta_0.shape, ths.shape, th0s.shape))

    for t in range(T):
      for i in range(n):
        y = labels[0,i]
        x = data[:,i]

        a = np.dot(x,theta)+theta_0
        #print("a = {}".format(a))
        positive = np.sign(y*a)

        if np.sign(y*a) <=0: # update the thetas
          theta[:,0] = theta[:,0]+ y*x
          theta_0 = theta_0 + y
        ths = ths + theta
        th0s = th0s + theta_0

    print("shape x = {}, y = {}, theta = {}, theta_0 = {}".format(x.shape,y.shape,theta.shape,theta_0.shape))
    return (ths/(n*T), th0s/(n*T))


In [10]:
test_averaged_perceptron(averaged_perceptron)

d = 2, n = 4, theta shape = (2, 1), theta_0 shape = (1,), ths shape = (2, 1), th0s = (1,)
shape x = (2,), y = (), theta = (2, 1), theta_0 = (1,)
-----------Test Averaged Perceptron 0-----------
Passed! 

d = 2, n = 4, theta shape = (2, 1), theta_0 shape = (1,), ths shape = (2, 1), th0s = (1,)
shape x = (2,), y = (), theta = (2, 1), theta_0 = (1,)
-----------Test Averaged Perceptron 1-----------
Passed! 



# 9) Implement evaluation strategies
  
## 9.1)  Evaluating a classifier

To evaluate a classifier, we are interested in how well it performs on data that it wasn't trained on. Construct a testing procedure that uses a training data set, calls a learning algorithm to get a linear separator (a tuple of $\theta, \theta_0$), and then reports the percentage correct on a new testing set as a float between 0. and 1..

The learning algorithm is passed as a function that takes a data array and a labels vector.  Your evaluator should be able to interchangeably evaluate `perceptron` or `averaged_perceptron` (or future algorithms with the same spec), depending on what is passed through the `learner` parameter.

The `eval_classifier` function should accept the following parameters:

* <tt>learner</tt> - a function, such as perceptron or averaged_perceptron
* <tt>data_train</tt> - training data
* <tt>labels_train</tt> - training labels
* <tt>data_test</tt> - test data
* <tt>labels_test</tt> - test labels

Assume that you have available the function `score` from HW 1, which takes inputs:

* <tt>data</tt>: a <tt>d</tt> by <tt>n</tt> array of floats (representing <tt>n</tt> data points in <tt>d</tt> dimensions)
* <tt>labels</tt>: a <tt>1</tt> by <tt>n</tt> array of elements in <tt>(+1, -1)</tt>, representing target labels
* <tt>th</tt>: a <tt>d</tt> by <tt>1</tt> array of floats that together with
* <tt>th0</tt>: a single scalar or 1 by 1 array, represents a hyperplane

and returns 1 by 1 matrix with an integer indicating number of data points correct for the separator.

In [57]:
def eval_classifier(learner, data_train, labels_train, data_test, labels_test):
  theta,theta_0 = learner(data_train,labels_train)
  print("theta shape = {}, theta_0 shape = {}, data_test shape = {}".format(theta.shape, theta_0.shape, data_test.shape))
  print(theta)
  raw_score = score(data_test,labels_test,theta,theta_0)
  d, test_n = data_test.shape
  return raw_score / test_n

In [ ]:
test_eval_classifier(eval_classifier,perceptron)

d = 2, n = 30, theta shape = (2, 1), theta_0 shape = (1,)
shape x = (2,), y = (), theta = (2, 1), theta_0 = (1,)
theta shape = (2, 1), theta_0 shape = (1,), data_test shape = (2, 30)
[[3.19695273]
 [0.10839974]]
-----------Test Eval Classifier 0-----------
Passed! 

d = 2, n = 30, theta shape = (2, 1), theta_0 shape = (1,)
shape x = (2,), y = (), theta = (2, 1), theta_0 = (1,)
theta shape = (2, 1), theta_0 shape = (1,), data_test shape = (2, 30)
[[-2.53272084]
 [ 2.5282913 ]]
-----------Test Eval Classifier 1-----------
Passed! 





```
# This is formatted as code
```

## <subsection>9.2) Evaluating a learning algorithm using a data source</subsection>

Construct a testing procedure that takes a learning algorithm and a data source as input and runs the learning algorithm multiple times, each time evaluating the resulting classifier as above. It should report the overall average classification accuracy.

You can use our implementation of `eval_classifier` as above.

Write the function `eval_learning_alg` that takes:

* <tt>learner</tt> - a function, such as perceptron or averaged_perceptron
* <tt>data_gen</tt> - a data generator, call it with a desired data set size; returns a tuple (data, labels)
* <tt>n_train</tt> - the size of the learning sets
* <tt>n_test</tt> - the size of the test sets
* <tt>it</tt> - the number of iterations to average over

and returns the average classification accuracy as a float between 0. and 1..  

** Note: Be sure to generate your training data and then testing data in that order, to ensure that the pseudorandomly generated data matches that in the test code. **

In [64]:
def eval_learning_alg(learner, data_gen, n_train, n_test, it):
    score = 0
    for i in range(it):
      data_train, labels_train = data_gen(n_train)
      data_test, labels_test = data_train, labels_train
      score += eval_classifier(learner, data_train, labels_train, data_test, labels_test)
    return score / it

In [ ]:
test_eval_learning_alg(eval_learning_alg,perceptron)

d = 2, n = 10, theta shape = (2, 1), theta_0 shape = (1,)
shape x = (2,), y = (), theta = (2, 1), theta_0 = (1,)
theta shape = (2, 1), theta_0 shape = (1,), data_test shape = (2, 10)
[[0.13368101]
 [0.87537888]]
d = 2, n = 10, theta shape = (2, 1), theta_0 shape = (1,)
shape x = (2,), y = (), theta = (2, 1), theta_0 = (1,)
theta shape = (2, 1), theta_0 shape = (1,), data_test shape = (2, 10)
[[-0.40522929]
 [-3.71502634]]
d = 2, n = 10, theta shape = (2, 1), theta_0 shape = (1,)
shape x = (2,), y = (), theta = (2, 1), theta_0 = (1,)
theta shape = (2, 1), theta_0 shape = (1,), data_test shape = (2, 10)
[[-3.27116076]
 [ 0.41978856]]
d = 2, n = 10, theta shape = (2, 1), theta_0 shape = (1,)
shape x = (2,), y = (), theta = (2, 1), theta_0 = (1,)
theta shape = (2, 1), theta_0 shape = (1,), data_test shape = (2, 10)
[[5.42252635]
 [1.80439054]]
d = 2, n = 10, theta shape = (2, 1), theta_0 shape = (1,)
shape x = (2,), y = (), theta = (2, 1), theta_0 = (1,)
theta shape = (2, 1), theta_0 shape

## <subsection>9.3) Evaluating a learning algorithm with a fixed dataset</subsection>

Cross-validation is a strategy for evaluating a learning algorithm, using a single training set of size $n$. Cross-validation takes in a learning algorithm $L$, a fixed data set $\mathcal{D}$, and a parameter $k$. It will run the learning algorithm $k$ different times, then evaluate the accuracy of the resulting classifier, and ultimately return the average of the accuracies over each of the $k$ "runs" of $L$. It is structured like this:

<pre><code>divide D into k parts, as equally as possible;  call them D_i for i == 0 .. k-1
# be sure the data is shuffled in case someone put all the positive examples first in the data!
for j from 0 to k-1:
    D_minus_j = union of all the datasets D_i, except for D_j
    h_j = L(D_minus_j)
    score_j = accuracy of h_j measured on D_j
return average(score0, ..., score(k-1))
</code></pre>

So, each time, it trains on  $k−1$ of the pieces of the data set and tests the resulting hypothesis on the piece that was not used for training.

When $k=n$, it is called *leave-one-out cross validation*.

Implement cross validation **assuming that the input data is shuffled already** so that the positives and negatives are distributed randomly. If the size of the data does not evenly divide by k, split the data into n % k sub-arrays of size n//k + 1 and the rest of size n//k. (Hint: You can use <a href="https://docs.scipy.org/doc/numpy/reference/generated/numpy.array_split.html">numpy.array_split</a>
and <a href="https://docs.scipy.org/doc/numpy/reference/generated/numpy.concatenate.html">numpy.concatenate</a> with axis arguments to split and rejoin the data as you desire.)

Note: In Python, n//k indicates integer division, e.g. 2//3 gives 0 and 4//3 gives 1.

In [44]:
def xval_learning_alg(learner, data, labels, k):
    data_split = np.array_split(data, k, axis=1)
    labels_split = np.array_split(labels, k, axis=1)

    print("k = {}, len of split = {}".format(k,len(data)))
    scores = []
    for i in range(k):
      data_minus_i = np.concatenate(data_split[:i] + data_split[i+1:],axis=1)
      labels_minus_i = np.concatenate(labels_split[:i] + labels_split[i+1:],axis=1)
      data_test = np.array(data_split[i])
      labels_test = np.array(labels_split[i])
      scores.append(eval_classifier(learner, data_minus_i, labels_minus_i, data_test,labels_test))

    return np.average(scores)


In [45]:
test_xval_learning_alg(xval_learning_alg,perceptron)

k = 5, len of split = 2
d = 2, n = 80, theta shape = (2, 1), theta_0 shape = (1,)
shape x = (2,), y = (), theta = (2, 1), theta_0 = (1,)
theta shape = (2, 1), theta_0 shape = (1,), data_test shape = (2, 20)
[[ 4.33750906]
 [-1.43826341]]
d = 2, n = 80, theta shape = (2, 1), theta_0 shape = (1,)
shape x = (2,), y = (), theta = (2, 1), theta_0 = (1,)
theta shape = (2, 1), theta_0 shape = (1,), data_test shape = (2, 20)
[[5.41400844]
 [0.12577919]]
d = 2, n = 80, theta shape = (2, 1), theta_0 shape = (1,)
shape x = (2,), y = (), theta = (2, 1), theta_0 = (1,)
theta shape = (2, 1), theta_0 shape = (1,), data_test shape = (2, 20)
[[4.96967703]
 [0.53961321]]
d = 2, n = 80, theta shape = (2, 1), theta_0 shape = (1,)
shape x = (2,), y = (), theta = (2, 1), theta_0 = (1,)
theta shape = (2, 1), theta_0 shape = (1,), data_test shape = (2, 20)
[[4.8554351 ]
 [1.04411716]]
d = 2, n = 80, theta shape = (2, 1), theta_0 shape = (1,)
shape x = (2,), y = (), theta = (2, 1), theta_0 = (1,)
theta shape =

## 10) Testing

In this section, we compare the effectiveness of perceptron and averaged perceptron on some data that are not necessarily linearly separable.

Use your `eval_learning_alg` and the `gen_flipped_lin_separable` generator in the code file to evaluate the accuracy of `perceptron` vs. a`veraged_perceptron`. `gen_flipped_lin_separable` can be called with an integer to return a data set and labels. Note that this generates linearly separable data and then "flips" the labels with some specified probability (the argument pflip); so most of the results will not be linearly separable. You can also specifiy pflip in the call to the generator. You should use the default values of th and th_0 to retain consistency with the Tutor.

Run enough trials so that you can confidently predict the accuracy of these algorithms on new data from that same generator; assume training/test sets on the order of 20 points. The Tutor will check that your answer is within 0.025 of the answer we got using the same generator.

In [68]:
print(eval_learning_alg(averaged_perceptron, gen_flipped_lin_separable(pflip=.25), 20, 20,50))

d = 2, n = 20, theta shape = (2, 1), theta_0 shape = (1,), ths shape = (2, 1), th0s = (1,)
shape x = (2,), y = (), theta = (2, 1), theta_0 = (1,)
theta shape = (2, 1), theta_0 shape = (1,), data_test shape = (2, 20)
[[0.65559284]
 [2.51604143]]
d = 2, n = 20, theta shape = (2, 1), theta_0 shape = (1,), ths shape = (2, 1), th0s = (1,)
shape x = (2,), y = (), theta = (2, 1), theta_0 = (1,)
theta shape = (2, 1), theta_0 shape = (1,), data_test shape = (2, 20)
[[0.86224574]
 [1.45343974]]
d = 2, n = 20, theta shape = (2, 1), theta_0 shape = (1,), ths shape = (2, 1), th0s = (1,)
shape x = (2,), y = (), theta = (2, 1), theta_0 = (1,)
theta shape = (2, 1), theta_0 shape = (1,), data_test shape = (2, 20)
[[0.29991919]
 [0.2218683 ]]
d = 2, n = 20, theta shape = (2, 1), theta_0 shape = (1,), ths shape = (2, 1), th0s = (1,)
shape x = (2,), y = (), theta = (2, 1), theta_0 = (1,)
theta shape = (2, 1), theta_0 shape = (1,), data_test shape = (2, 20)
[[3.24869563]
 [3.99411383]]
d = 2, n = 20, theta